In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


#print(pd.__version__)

dataframe = pd.read_csv('health_data.csv')
print(dataframe.head())
print(dataframe.info())

# Features for the model
features = ['heart_rate', 'activity_level', 'age']

# Extract the features from the data frame
X = dataframe[features]
print(X.head())

# Identify the label
label = 'risk_level'

# Extract the label from the data frame 
y = dataframe[label]
print(y.head())

# Split the data into training and testing sets 
# Set the split ratio
test_size = 0.2

# random_state for reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=22)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)













   heart_rate  activity_level  age  risk_level
0          86               8   48           0
1          63               8   26           0
2          76               8   34           1
3          78               7   53           1
4          94               8   35           0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   heart_rate      500 non-null    int64
 1   activity_level  500 non-null    int64
 2   age             500 non-null    int64
 3   risk_level      500 non-null    int64
dtypes: int64(4)
memory usage: 15.8 KB
None
   heart_rate  activity_level  age
0          86               8   48
1          63               8   26
2          76               8   34
3          78               7   53
4          94               8   35
0    0
1    0
2    1
3    1
4    0
Name: risk_level, dtype: int64
(400, 3)
(100, 3)
(400,)
(100,)
